In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import math

import numpy as np
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib import cm
import os
import re # para remplazar puntos  pop = re.sub(r'[^\w]','-',pop)     si tiene pop  puntos los cambia por -
from numpy import nan
  
from fuzzywuzzy import fuzz  
import unicodedata
#str.isnumeric () #si una cadena es un número entero positivo
#str.isdecimal() #Determina si una cadena es un decimal
#str.isalpha() # determina si la cadena son solo alfabética
#str.isdigit()  #Determina si una cadena es un dígito decimal
#str.isalnum() #Determinar si la cadena es alfanumérica
#str.isascii()#Determina si las cadenas son caracteres ASCII: 

carga los modulos

In [2]:
from Pmod_1  import primer_modulo_1 as mod_1
from Pmod_1.primer_modulo_1  import IDs
from Pmod_1.primer_modulo_1  import Nombres_1_X2

In [3]:
Nombres_1_X2

,0,1
0,Nom_mun,0
1,Nom_ent,0
2,DL_Cabecera,0
3,DF_Cabecera,0


In [4]:
IDs

,0,1
0,entidad,0
1,seccion,0
2,municipio,0
3,distrito_l,0
4,distrito_f,0


# solo es valido para Precidencial , gobernador, un solo ayuntamiento y un solo distrito 

se eleije los datos de la eleccion, deben ser los mismos que se usaron en P1-generador de carpetas.

In [5]:
from imagenes  import graficas_base_1 as graficas_base_1
from imagenes  import Datos
from imagenes.Datos  import estado_XX_id

In [6]:
tipo_eleccion='Municipal'  # de que fue la eleccion 
#tipo_eleccion="Alcaldia"
#tipo_eleccion='Diputados Federales' 
#tipo_eleccion='Diputados Locales'
#tipo_eleccion='Presidencial'  
#tipo_eleccion='Gobernatura' 
#tipo_eleccion='Senaduria'

#clave_entidad=15
#tipo_eleccion='Gobernatura'  # de que fue la eleccion 
año='2024'                  #año de la eleccion

#entidad=estado_XX_id.loc[clave_entidad,'NOMBRE_ESTADO']
#entidad2=estado_XX_id.loc[clave_entidad,'NOMBRE_ESTADO_2']
rut='/Users/ds/Documents/Base_municipales_200924'    #Ruta donde se guardan las bases

carga de datos de ventiles

In [7]:
gdf1 = gpd.read_file("/Users/ds/Documents/carpeta sin título/ventiles/Ventiles 2020/Secciones_2020INE_Ventiles.shp")   # infromacion de lso ventiles 
gdf1=gdf1[['entidad','seccion','ventilN','ventilL']]
#gdf1=gdf1[gdf1['entidad']==clave_entidad]

# para cargar datos csv

In [8]:
Ruta_base_prep=Ruta_base_prep="/Users/ds/Documents/base_electoral_18_sep_24/Sinaloa/SIN_AYUNTAMIENTO_2024.csv"

In [9]:
prep_AA=Ruta_base_prep.split('/')[-2] # solo sirve para poner nombre a la base 


In [10]:
df11T = pd.read_csv(Ruta_base_prep, header=0, sep=",")  # Carga los datos
#df11T=pd.read_csv(Ruta_base_prep, header=6)
df11T


,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO_LOCAL,CABECERA_DISTRITAL_LOCAL,ID_MUNICIPIO_LOCAL,MUNICIPIO_LOCAL,SECCION,TIPO_CASILLA,PAN,PRI,...,C_PAN_PRI,C_PAN_PRD,C_PAN_PAS,C_PRI_PRD,C_PRI_PAS,C_PRD_PAS,CC_PVEM_MORENA,VCN,VN,TOTAL_VOTOS
0,25,SINALOA,4,LOS MOCHIS,1,AHOME,MESA 1,VA,1,1,...,0,0,0,0,0,0,\N,0,1,16
1,25,SINALOA,5,LOS MOCHIS,1,AHOME,2,B,29,17,...,0,0,0,0,0,0,\N,0,7,156
2,25,SINALOA,5,LOS MOCHIS,1,AHOME,3,B,120,92,...,4,0,0,0,0,0,\N,0,6,447
3,25,SINALOA,5,LOS MOCHIS,1,AHOME,4,B,40,46,...,1,0,0,0,1,0,\N,0,14,259
4,25,SINALOA,5,LOS MOCHIS,1,AHOME,5,B,29,41,...,1,1,0,0,0,0,\N,0,21,263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5169,25,SINALOA,6,MOCORITO,17,SINALOA,3609,B,10,22,...,\N,\N,\N,\N,\N,\N,\N,0,21,224
5170,25,SINALOA,6,MOCORITO,17,SINALOA,3610,B,9,20,...,\N,\N,\N,\N,\N,\N,\N,0,23,333
5171,25,SINALOA,6,MOCORITO,17,SINALOA,3611,B,15,27,...,\N,\N,\N,\N,\N,\N,\N,0,26,263
5172,25,SINALOA,6,MOCORITO,17,SINALOA,3612,B,2,18,...,\N,\N,\N,\N,\N,\N,\N,0,14,150


In [11]:
df11T.columns

Index(['ID_ESTADO', 'NOMBRE_ESTADO', 'ID_DISTRITO_LOCAL',
       'CABECERA_DISTRITAL_LOCAL', 'ID_MUNICIPIO_LOCAL', 'MUNICIPIO_LOCAL',
       'SECCION', 'TIPO_CASILLA', 'PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC',
       'PAS', 'MORENA', 'PES', 'MAG', 'JSLS', 'VMSA', 'C_PAN_PRI_PRD_PAS',
       'C_PAN_PRI_PRD', 'C_PAN_PRI_PAS', 'C_PAN_PRD_PAS', 'C_PRI_PRD_PAS',
       'C_PAN_PRI', 'C_PAN_PRD', 'C_PAN_PAS', 'C_PRI_PRD', 'C_PRI_PAS',
       'C_PRD_PAS', 'CC_PVEM_MORENA', 'VCN', 'VN', 'TOTAL_VOTOS'],
      dtype='object')

In [12]:
#df11T=df11T[df11T['CONTABILIZADA']=='1'] #solo en prep
#df11T=df11T[df11T['CONTABILIZADA']==1]
#df11T

Se ponen los IDs de la entidad estas columnas son numeros

In [13]:
IDs_entidad=['ID_ESTADO']

IDs_seccion=['SECCION']

IDs_municipio=['ID_MUNICIPIO_LOCAL']

IDs_distrito_l=[]

IDs_distrito_f=[]

nombres de los ids entidad


In [14]:
Nombre_municipio=[ 'MUNICIPIO_LOCAL']
Nombre_entidad=['NOMBRE_ESTADO']
Nombre_Cabecera_distrito_local=[]
Nombre_Cabecera_distrito_federal=[]


# Partidos Politicos

aqui  se ponen solo los partidos politicos, además las coaliciones conjuntas(cuando aparece una sola opcion en la boleta ). A las coaliciones conjuntas se deben separar por - ó _  ejemplo además si tienen una C o CC o cM al principio o final no afecta ejempli CC_PAN_PRI_PRD

los partidos Politicos se ponen en mayusculas en espcial en todo el pais esta homlogado en nuestra bases ( PAN PRI PRD MORENA PT PVEM PRD) los demas partidos politicos 

In [15]:
df11T.columns 

Index(['ID_ESTADO', 'NOMBRE_ESTADO', 'ID_DISTRITO_LOCAL',
       'CABECERA_DISTRITAL_LOCAL', 'ID_MUNICIPIO_LOCAL', 'MUNICIPIO_LOCAL',
       'SECCION', 'TIPO_CASILLA', 'PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC',
       'PAS', 'MORENA', 'PES', 'MAG', 'JSLS', 'VMSA', 'C_PAN_PRI_PRD_PAS',
       'C_PAN_PRI_PRD', 'C_PAN_PRI_PAS', 'C_PAN_PRD_PAS', 'C_PRI_PRD_PAS',
       'C_PAN_PRI', 'C_PAN_PRD', 'C_PAN_PAS', 'C_PRI_PRD', 'C_PRI_PAS',
       'C_PRD_PAS', 'CC_PVEM_MORENA', 'VCN', 'VN', 'TOTAL_VOTOS'],
      dtype='object')

Reneombro a los candidatos independientes

In [16]:
candidatos_indwpwndientes_1=['CAND_IND']

In [17]:
candidatos_indwpwndientes_1_rename=['CAND_IND_1']

In [18]:
for n in range(0,len(candidatos_indwpwndientes_1)):
    df11T=df11T.rename({candidatos_indwpwndientes_1[n]:candidatos_indwpwndientes_1_rename[n]},axis=1)

In [19]:
#df11T.iloc[:,10:]

In [20]:
df11T.columns

Index(['ID_ESTADO', 'NOMBRE_ESTADO', 'ID_DISTRITO_LOCAL',
       'CABECERA_DISTRITAL_LOCAL', 'ID_MUNICIPIO_LOCAL', 'MUNICIPIO_LOCAL',
       'SECCION', 'TIPO_CASILLA', 'PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC',
       'PAS', 'MORENA', 'PES', 'MAG', 'JSLS', 'VMSA', 'C_PAN_PRI_PRD_PAS',
       'C_PAN_PRI_PRD', 'C_PAN_PRI_PAS', 'C_PAN_PRD_PAS', 'C_PRI_PRD_PAS',
       'C_PAN_PRI', 'C_PAN_PRD', 'C_PAN_PAS', 'C_PRI_PRD', 'C_PRI_PAS',
       'C_PRD_PAS', 'CC_PVEM_MORENA', 'VCN', 'VN', 'TOTAL_VOTOS'],
      dtype='object')

In [21]:
Partidos=['PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC',
       'PAS', 'MORENA', 'PES', 'MAG', 'JSLS', 'VMSA', 'CC_PVEM_MORENA']

las coalciones deben estar separadas por _ además de que las siglas que la conforman deben ser las mismas de la lista que se creo arriba. de esta forma a la hora de romper la cadena podemos subtraer la informacion de las columans que esta compuesta

In [22]:
coaliciones_RT=['C_PAN_PRI_PRD_PAS',
       'C_PAN_PRI_PRD', 'C_PAN_PRI_PAS', 'C_PAN_PRD_PAS', 'C_PRI_PRD_PAS',
       'C_PAN_PRI', 'C_PAN_PRD', 'C_PAN_PAS', 'C_PRI_PRD', 'C_PRI_PAS',
       'C_PRD_PAS']

los candidatos independientes se deben renombrar a CAND_IND enb su primera letras despues se puede poner como viene en la base de datos ejemplo si viene CPRTY se renombra a CAND_IND_CPRTY u otra opcion seriua CAND_IND_1 si no hay la lista puede quedar vacia

In [23]:
candidatos_indepentiens=[]


DAtos de votos nulos y candidatos no registrados. Estas columans son escenciales hasta ahora siemore se an encontrado en las bases de datos si no llegaran a existir y estamos seguros que no hubo vatos de candidatos no registrados y de votos nulos se crea la columna con 0

las columans deben tener el nombre  de 'NUM_VOTOS_CAN_NREG','NUM_VOTOS_NULOS' respectivamente

In [24]:
df11T.columns

Index(['ID_ESTADO', 'NOMBRE_ESTADO', 'ID_DISTRITO_LOCAL',
       'CABECERA_DISTRITAL_LOCAL', 'ID_MUNICIPIO_LOCAL', 'MUNICIPIO_LOCAL',
       'SECCION', 'TIPO_CASILLA', 'PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC',
       'PAS', 'MORENA', 'PES', 'MAG', 'JSLS', 'VMSA', 'C_PAN_PRI_PRD_PAS',
       'C_PAN_PRI_PRD', 'C_PAN_PRI_PAS', 'C_PAN_PRD_PAS', 'C_PRI_PRD_PAS',
       'C_PAN_PRI', 'C_PAN_PRD', 'C_PAN_PAS', 'C_PRI_PRD', 'C_PRI_PAS',
       'C_PRD_PAS', 'CC_PVEM_MORENA', 'VCN', 'VN', 'TOTAL_VOTOS'],
      dtype='object')

In [25]:
df11T=df11T.rename({'VCN': 'NUM_VOTOS_CAN_NREG',
                    'VN':'NUM_VOTOS_NULOS',
                    'LISTA_NOMINAL': 'LISTA_NOMINAL'
                    },axis=1)

In [26]:
NUM_VOTOS_CAN_NREG=['NUM_VOTOS_CAN_NREG']
NUM_VOTOS_NULOS=['NUM_VOTOS_NULOS']

In [27]:
print(len(df11T["SECCION"].unique()))

3793


# Usando la base de lista nominal

In [28]:
ruta_nominal="/Users/ds/Documents/Lista_Nominal_250924/Lista_Nominal_Pais_250924.csv"

lis_no=pd.read_csv(ruta_nominal, header=0)
lis_no.columns

Index(['entidad', 'Nom_ent', 'distrito_f', 'DF_Cabecera', 'municipio',
       'Nom_mun', 'seccion', 'Lista Nominal'],
      dtype='object')

In [29]:
lis_no[(lis_no['entidad']==25)&(lis_no['municipio']==1)&(lis_no['seccion']>0)]['Lista Nominal'].sum()

350652

In [30]:
lis_no[(lis_no['municipio']==1)&(lis_no['seccion']>0)]['Lista Nominal'].sum()

3611724

In [31]:
lis_nominal=lis_no[lis_no["entidad"]==25]

In [32]:
lis_nominal

,entidad,Nom_ent,distrito_f,DF_Cabecera,municipio,Nom_mun,seccion,Lista Nominal
54266,25,RESIDENTES EXTRANJERO,0,0,0,0,0,13194
54267,25,SINALOA,1,MAZATLAN ...,4,CONCORDIA,629,598
54268,25,SINALOA,1,MAZATLAN ...,4,CONCORDIA,630,308
54269,25,SINALOA,1,MAZATLAN ...,4,CONCORDIA,631,790
54270,25,SINALOA,1,MAZATLAN ...,4,CONCORDIA,632,1204
...,...,...,...,...,...,...,...,...
58090,25,SINALOA,7,CULIACAN ...,6,CULIACAN,3993,3090
58091,25,SINALOA,7,CULIACAN ...,6,CULIACAN,3994,1915
58092,25,SINALOA,7,CULIACAN ...,6,CULIACAN,3995,1120
58093,25,SINALOA,7,CULIACAN ...,6,CULIACAN,3996,1762


In [37]:
lis_nominal[(lis_nominal['municipio']==1)&(lis_nominal['seccion']>0)]['Lista Nominal'].sum()

350652

In [38]:
df11T.columns

Index(['ID_ESTADO', 'NOMBRE_ESTADO', 'ID_DISTRITO_LOCAL',
       'CABECERA_DISTRITAL_LOCAL', 'ID_MUNICIPIO_LOCAL', 'MUNICIPIO_LOCAL',
       'SECCION', 'TIPO_CASILLA', 'PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC',
       'PAS', 'MORENA', 'PES', 'MAG', 'JSLS', 'VMSA', 'C_PAN_PRI_PRD_PAS',
       'C_PAN_PRI_PRD', 'C_PAN_PRI_PAS', 'C_PAN_PRD_PAS', 'C_PRI_PRD_PAS',
       'C_PAN_PRI', 'C_PAN_PRD', 'C_PAN_PAS', 'C_PRI_PRD', 'C_PRI_PAS',
       'C_PRD_PAS', 'CC_PVEM_MORENA', 'NUM_VOTOS_CAN_NREG', 'NUM_VOTOS_NULOS',
       'TOTAL_VOTOS'],
      dtype='object')

In [39]:
lis_nominal= lis_nominal[['entidad', 'Nom_ent', 'municipio',
       'Nom_mun', 'seccion', 'Lista Nominal']]


lis_nominal=lis_nominal.rename(columns={'entidad': 'ID_ESTADO'
                            , 'Nom_ent': 'NOMBRE_ESTADO'
                            , 'municipio':'ID_MUNICIPIO_LOCAL'
                            ,'Nom_mun': 'MUNICIPIO_LOCAL'
                            , 'seccion': 'SECCION'
                            , 'Lista Nominal': 'LISTA_NOMINAL'
    
})
lis_nominal.columns

Index(['ID_ESTADO', 'NOMBRE_ESTADO', 'ID_MUNICIPIO_LOCAL', 'MUNICIPIO_LOCAL',
       'SECCION', 'LISTA_NOMINAL'],
      dtype='object')

In [40]:
df11T_1=df11T.copy()
lis_nominal_1= lis_nominal.copy()



In [67]:
df11T_1["SECCION"]=df11T_1["SECCION"].replace("MESA", "", regex=True)
df11T_1[['ID_MUNICIPIO_LOCAL','SECCION']]=df11T_1[['ID_MUNICIPIO_LOCAL','SECCION']].astype("int")
lis_nominal_1[['ID_MUNICIPIO_LOCAL','SECCION', 'LISTA_NOMINAL']]=lis_nominal_1[['ID_MUNICIPIO_LOCAL','SECCION', 'LISTA_NOMINAL']].astype("int")
df11T_1.dtypes

ID_ESTADO                    int64
NOMBRE_ESTADO               object
ID_DISTRITO_LOCAL            int64
CABECERA_DISTRITAL_LOCAL    object
ID_MUNICIPIO_LOCAL           int64
MUNICIPIO_LOCAL             object
SECCION                      int64
TIPO_CASILLA                object
PAN                         object
PRI                         object
PRD                         object
PT                          object
PVEM                        object
MC                          object
PAS                         object
MORENA                      object
PES                         object
MAG                         object
JSLS                        object
VMSA                        object
C_PAN_PRI_PRD_PAS           object
C_PAN_PRI_PRD               object
C_PAN_PRI_PAS               object
C_PAN_PRD_PAS               object
C_PRI_PRD_PAS               object
C_PAN_PRI                   object
C_PAN_PRD                   object
C_PAN_PAS                   object
C_PRI_PRD           

In [68]:
lis_nominal_1.dtypes

ID_ESTADO              int64
NOMBRE_ESTADO         object
ID_MUNICIPIO_LOCAL     int64
MUNICIPIO_LOCAL       object
SECCION                int64
LISTA_NOMINAL          int64
dtype: object

In [69]:
lis_nominal_1.columns

Index(['ID_ESTADO', 'NOMBRE_ESTADO', 'ID_MUNICIPIO_LOCAL', 'MUNICIPIO_LOCAL',
       'SECCION', 'LISTA_NOMINAL'],
      dtype='object')

In [108]:
union=pd.merge(df11T_1, lis_nominal_1[['ID_MUNICIPIO_LOCAL','SECCION']], how="inner", on=['ID_MUNICIPIO_LOCAL','SECCION'])

#union= union.drop_duplicates()
union

,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO_LOCAL,CABECERA_DISTRITAL_LOCAL,ID_MUNICIPIO_LOCAL,MUNICIPIO_LOCAL,SECCION,TIPO_CASILLA,PAN,PRI,...,C_PAN_PRI,C_PAN_PRD,C_PAN_PAS,C_PRI_PRD,C_PRI_PAS,C_PRD_PAS,CC_PVEM_MORENA,NUM_VOTOS_CAN_NREG,NUM_VOTOS_NULOS,TOTAL_VOTOS
0,25,SINALOA,5,LOS MOCHIS,1,AHOME,2,B,29,17,...,0,0,0,0,0,0,\N,0,7,156
1,25,SINALOA,5,LOS MOCHIS,1,AHOME,3,B,120,92,...,4,0,0,0,0,0,\N,0,6,447
2,25,SINALOA,5,LOS MOCHIS,1,AHOME,4,B,40,46,...,1,0,0,0,1,0,\N,0,14,259
3,25,SINALOA,5,LOS MOCHIS,1,AHOME,5,B,29,41,...,1,1,0,0,0,0,\N,0,21,263
4,25,SINALOA,5,LOS MOCHIS,1,AHOME,6,B,17,13,...,0,0,1,0,0,0,\N,0,10,148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089,25,SINALOA,6,MOCORITO,17,SINALOA,3609,B,10,22,...,\N,\N,\N,\N,\N,\N,\N,0,21,224
5090,25,SINALOA,6,MOCORITO,17,SINALOA,3610,B,9,20,...,\N,\N,\N,\N,\N,\N,\N,0,23,333
5091,25,SINALOA,6,MOCORITO,17,SINALOA,3611,B,15,27,...,\N,\N,\N,\N,\N,\N,\N,0,26,263
5092,25,SINALOA,6,MOCORITO,17,SINALOA,3612,B,2,18,...,\N,\N,\N,\N,\N,\N,\N,0,14,150


In [109]:
print(len(lis_nominal_1["SECCION"].unique()))
print(len(df11T_1["SECCION"].unique()))

3829
3791


In [110]:
union[union["SECCION"]==0]

,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO_LOCAL,CABECERA_DISTRITAL_LOCAL,ID_MUNICIPIO_LOCAL,MUNICIPIO_LOCAL,SECCION,TIPO_CASILLA,PAN,PRI,...,C_PAN_PRI,C_PAN_PRD,C_PAN_PAS,C_PRI_PRD,C_PRI_PAS,C_PRD_PAS,CC_PVEM_MORENA,NUM_VOTOS_CAN_NREG,NUM_VOTOS_NULOS,TOTAL_VOTOS


In [111]:
union[(union['ID_MUNICIPIO_LOCAL']==1)&(union['SECCION']>0)]['LISTA_NOMINAL'].sum()

KeyError: 'LISTA_NOMINAL'

In [112]:
lis_nominal_1[(lis_nominal_1['ID_MUNICIPIO_LOCAL']==1)&(lis_nominal_1['SECCION']>0)]['LISTA_NOMINAL'].sum()

350652

In [76]:
df11T=union
df11T

,ID_ESTADO,NOMBRE_ESTADO,ID_DISTRITO_LOCAL,CABECERA_DISTRITAL_LOCAL,ID_MUNICIPIO_LOCAL,MUNICIPIO_LOCAL,SECCION,TIPO_CASILLA,PAN,PRI,...,C_PAN_PRD,C_PAN_PAS,C_PRI_PRD,C_PRI_PAS,C_PRD_PAS,CC_PVEM_MORENA,NUM_VOTOS_CAN_NREG,NUM_VOTOS_NULOS,TOTAL_VOTOS,LISTA_NOMINAL
0,25,SINALOA,5,LOS MOCHIS,1,AHOME,2,B,29,17,...,0,0,0,0,0,\N,0,7,156,254
1,25,SINALOA,5,LOS MOCHIS,1,AHOME,3,B,120,92,...,0,0,0,0,0,\N,0,6,447,657
2,25,SINALOA,5,LOS MOCHIS,1,AHOME,4,B,40,46,...,0,0,0,1,0,\N,0,14,259,355
3,25,SINALOA,5,LOS MOCHIS,1,AHOME,5,B,29,41,...,1,0,0,0,0,\N,0,21,263,432
4,25,SINALOA,5,LOS MOCHIS,1,AHOME,6,B,17,13,...,0,1,0,0,0,\N,0,10,148,241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089,25,SINALOA,6,MOCORITO,17,SINALOA,3609,B,10,22,...,\N,\N,\N,\N,\N,\N,0,21,224,385
5090,25,SINALOA,6,MOCORITO,17,SINALOA,3610,B,9,20,...,\N,\N,\N,\N,\N,\N,0,23,333,613
5091,25,SINALOA,6,MOCORITO,17,SINALOA,3611,B,15,27,...,\N,\N,\N,\N,\N,\N,0,26,263,420
5092,25,SINALOA,6,MOCORITO,17,SINALOA,3612,B,2,18,...,\N,\N,\N,\N,\N,\N,0,14,150,237


In [77]:
#df11T = df11T.dropna(subset=["MUNICIPIO_LOCAL_x"])
#df11T

In [78]:
df11T[(df11T['ID_MUNICIPIO_LOCAL']==1)&(df11T['SECCION']>0)]['LISTA_NOMINAL'].sum()

863666

In [47]:
wrrthsrt

NameError: name 'wrrthsrt' is not defined

In [38]:
df11T= df11T.drop(['ID_ESTADO_y', 'NOMBRE_ESTADO_y', 'MUNICIPIO_LOCAL_y'], axis=1)

In [ ]:
df11T.columns

In [40]:

df11T=df11T.rename(columns={'ID_ESTADO_x':'ID_ESTADO', 
                            'NOMBRE_ESTADO_x': 'NOMBRE_ESTADO',
                            'MUNICIPIO_LOCAL_x': "MUNICIPIO_LOCAL"

})

In [ ]:
df11T.columns

In [ ]:
parar

# Regresando al programa

In [48]:
Lista_nominal=['LISTA_NOMINAL']

el total de votos no es indispensable pero se agrega solo por si acaso

In [49]:

Total_votos=[] # no es indipensable


In [ ]:
df11T.columns

In [51]:
datos_entidad=mod_1.IDS_1(IDs,IDs_entidad,IDs_seccion,IDs_municipio,IDs_distrito_l,IDs_distrito_f)
nombresst1_2=mod_1.Nombres_1_X2_1(Nombres_1_X2,Nombre_municipio,Nombre_entidad,Nombre_Cabecera_distrito_local,Nombre_Cabecera_distrito_federal)
datoseleccion_1_2=mod_1.datoseleccion_1_2_1(Total_votos,Lista_nominal)
sinCoa1X_2=mod_1.sinCoa1X_2_1(NUM_VOTOS_CAN_NREG,NUM_VOTOS_NULOS)
Tatal_15,datos_enteros=mod_1.rename_1(df11T,datoseleccion_1_2,datos_entidad,nombresst1_2,sinCoa1X_2,Partidos,coaliciones_RT,candidatos_indepentiens)

In [ ]:
nombresst1_2


# aqui se hace limpieza de los datos 

In [ ]:
df11T[datos_enteros].dtypes

In [54]:
df11T=df11T.replace('Sin dato',0)
df11T=df11T.replace('Ilegible',0)
df11T=df11T.replace( '-',nan)

In [55]:
df11T=df11T.replace('Sin dato',0)
df11T=df11T.replace('ILEGIBLE ',0)
df11T=df11T.replace('ILEGIBLE',0)
df11T=df11T.replace('SIN DATO',0)
df11T=df11T.replace('Ilegible',0)
df11T=df11T.replace('SIN ACTA',0)

In [56]:
df11T=df11T.replace('=',"", regex=True)
df11T=df11T.replace('"',"", regex=True)

In [57]:
df11T=df11T.replace('\\N', 0)

In [ ]:
df11T

la lista datos_enteros deben ser enteroes

In [59]:
df11T[datos_enteros]=df11T[datos_enteros].fillna(0)
df11T[datos_enteros]=df11T[datos_enteros].astype('int')

In [60]:
ABC_grup=(list(pd.concat([datos_entidad[0],nombresst1_2[0]]).reset_index(drop=True)))
ABC_grup_2=pd.concat([pd.Series(Partidos, dtype='object'),pd.Series(coaliciones_RT, dtype='object'),pd.Series(candidatos_indepentiens, dtype='object'),sinCoa1X_2[0],datoseleccion_1_2[1]]).reset_index(drop=True)

In [ ]:
df11T[datos_enteros].dtypes

In [62]:
#------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------#
#procesamiento_1 genrera dos Series una con informacion de los nombres de los partidos y otra con informacion de partidos coaliciones y candidatos independientes ade mas agrupa la informacion por seccion electoral y la suma
        #ABC_grup es la informacion espacial de la elecciones(entidad seccion municipio nombre cabezera etc...)
                # siguiendo nuestra guia se puede generar con ABC_grup=(list(pd.concat([datos_entidad[0],nombresst1_2[0]]).reset_index(drop=True)))
        #ABC_grup_2 son todos los datos que numericos que se van a sumar Partidos coaliciones candidatos_indepentiens  inCoa1X_2  datoseleccion_1_2
                # siguiendo nuestra gria se puede generar  ABC_grup_2=pd.concat([pd.Series(Partidos, dtype='object'),pd.Series(coaliciones_RT, dtype='object'),pd.Series(candidatos_indepentiens, dtype='object'),sinCoa1X_2[0],datoseleccion_1_2[1]]).reset_index(drop=True)
def procesamiento_1(df11T,ABC_grup,ABC_grup_2,Partidos,coaliciones_RT,candidatos_indepentiens):
    df11T=df11T.groupby(ABC_grup,dropna=False)[ABC_grup_2].sum().reset_index()
    participantes_1_1= pd.Series(Partidos) 
    participantes_2_0_1= pd.concat([pd.Series(Partidos, dtype='object'),pd.Series(coaliciones_RT, dtype='object'),pd.Series(candidatos_indepentiens, dtype='object')]).reset_index(drop=True)
    return df11T,participantes_1_1,participantes_2_0_1
#------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------##------------#

In [ ]:
df11T[datos_enteros].dtypes

In [64]:
df11T,participantes_1_1,participantes_2_0_1=mod_1.procesamiento_1(df11T,ABC_grup,ABC_grup_2,Partidos,coaliciones_RT,candidatos_indepentiens)

In [65]:
tryu_XY,tryu_XY_2=mod_1.procesamiento_2(participantes_1_1,participantes_2_0_1)

In [66]:
participantes_2,participantes_2_XZ=mod_1.procesamiento_2_2(df11T,tryu_XY_2)

In [67]:
df11T,df11Ti_df22T,list3345=mod_1.procesamiento_2_2_3(df11T,tipo_eleccion,participantes_2)  

In [ ]:
df11Ti_df22T

In [69]:
df11Ti_df22T=df11Ti_df22T.replace(nan,0).sort_values("municipio",ascending=True)

Esta es la base que nos servira de guia para saber si estan bien las coalicioes si no se tiene esa base se utiliza  df11Ti_df22T pero se debe verificar que sean las coaliciones

In [70]:
#Ruta_candidaturas="/Users/ds/Documents/DL_Ayuntamientos/resultados_computos_2024/RESULTADOS COMPUTOS AYUNTAMIENTO.xlsx"

In [71]:
#dfPPTX_X =pd.read_excel(Ruta_candidaturas,header=5) # base de datos eleccion solo si se tiene los candidatos que participaron

In [72]:
dfPPTX_X=df11Ti_df22T

In [ ]:
dfPPTX_X.columns

In [ ]:
dfPPTX_X=dfPPTX_X[['municipio', 'PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC', 'PAS', 'MORENA',
       'PES', 'MAG', 'JSLS', 'VMSA', 'CC_PVEM_MORENA', 'C_PAN_PRI_PRD_PAS',
       'C_PAN_PRI_PRD', 'C_PAN_PRI_PAS', 'C_PAN_PRD_PAS', 'C_PRI_PRD_PAS',
       'C_PAN_PRI', 'C_PAN_PRD', 'C_PAN_PAS', 'C_PRI_PRD', 'C_PRI_PAS',
       'C_PRD_PAS']]
dfPPTX_X

vuelvo a cambiar el nombre las variables 

In [75]:
for n in range(0,len(candidatos_indwpwndientes_1)):
    dfPPTX_X=dfPPTX_X.rename({candidatos_indwpwndientes_1[n]:candidatos_indwpwndientes_1_rename[n]},axis=1)

In [ ]:
dfPPTX_X

Aqui es donde se puede poner df11Ti_df22T en lugar dfPPTX_X siempre recordondo que se deben de verificar los participanestes

In [77]:
dfPPT=dfPPTX_X.replace(nan,0).sort_values('municipio',ascending=True)
dfPPT=dfPPT.set_index('municipio')
tyup_l=dfPPT.columns
dfPPT=dfPPT.reset_index()


In [78]:
lpdfPPT=list(tyup_l) # tyup_l son las columnas de los participantes de las elecciones 
participantes_1_1_dfPPT= pd.DataFrame(lpdfPPT) 

In [79]:
# esta funcion ayuda a ver si las coluimnas se llaman de igual forma que la base de datos y de esta forma poder comprar 
#dfPPT   es la base de datos de los resultados por municipio o los participantes por municipio, solo necesita que sea un data frame con columnas entidad,distrito local, municipal o deferal 
        # los participantes(partidos o can ind) que no particpen en el municipio o distrito deben ir en nan o 
        # un ejemplo si en el mun 1 la coalcion C PAN PRD PRI tiene votos se pone 1 o un un numero mayor a 1 pero si en el municpìo 2 no hay C PAN PRI PRD se pone 0 

#pqw 
#participantes_2_XZ


def funayu1(dfPPT,participantes_2_XZ):
    pqw=pd.DataFrame(dfPPT.columns)
    dfPPT=dfPPT.replace('VMRE',0)  # remplaza el id de estado municipio o distritito de los los cotantes en el extranjero por un 0 puede cambiar'VMRE' por lo que se debe tener cuidado  
    f='ID_DISTRITO_LOCAL'
    for i_2 in range(0,len(pqw)):   
        b2=pqw.loc[i_2,0]
        if f in b2:
            f=b2
        c2=fuzz.token_sort_ratio(f,b2)
        if c2>88:
            dfPPT=dfPPT.rename({f:'ID_DISTRITO'},axis=1)
            pqw=pd.DataFrame(dfPPT.columns)
    f='ID_DISTRITO_FEDERAL'
    for i_2 in range(0,len(pqw)):   
        b2=pqw.loc[i_2,0]
        if f in b2:
            f=b2
        c2=fuzz.token_sort_ratio(f,b2)
        if c2>88:
            dfPPT=dfPPT.rename({f:'ID_DISTRITO'},axis=1)
            pqw=pd.DataFrame(dfPPT.columns)



    if tipo_eleccion=='Diputados Federales'  :
        datos_entidad_dfPPT=pd.DataFrame(['distrito_f'])
        datos_entidad_dfPPT[1]=pd.Series(['ID_DISTRITO'])

    if tipo_eleccion=='Diputados Locales':
        datos_entidad_dfPPT=pd.DataFrame(['distrito_l'])
        datos_entidad_dfPPT[1]=pd.Series(['ID_DISTRITO'])

    if tipo_eleccion=='Municipal'  :
        datos_entidad_dfPPT=pd.DataFrame(['municipio'])
        datos_entidad_dfPPT[1]=pd.Series([ 'ID_MUNICIPIO'])



    if tipo_eleccion=='Senaduria'  :
        datos_entidad_dfPPT=pd.DataFrame(['entidad'])
        datos_entidad_dfPPT[1]=pd.Series([ 'entidad'])
    datos_entidad_dfPPT[2]=0
    datos_entidad_dfPPT[2]=datos_entidad_dfPPT[2].astype('object')

    #datos_entidad_dfPPT 0 son los nom,bres que queremos y 2 son los que vienen en la base 1 son lo nombres más comunes

    for h in range(0,len(datos_entidad_dfPPT)):
        a=datos_entidad_dfPPT.loc[h,1]
        for i in range(0,len(pqw)):   
            b=pqw.loc[i,0]
            if a in b:
                a=b
            c=fuzz.token_sort_ratio(a,b)
            if c>85:
                datos_entidad_dfPPT.loc[h,2]=b
    for h in range(0,len(datos_entidad_dfPPT)):
            a=datos_entidad_dfPPT.loc[h,0]
            for i in range(0,len(pqw)):   
                b=pqw.loc[i,0]
                if a in b:
                    a=b
                c=fuzz.ratio(a,b)
                if c>85:
                    datos_entidad_dfPPT.loc[h,2]=b

                    
    for n in range (0,len(datos_entidad_dfPPT)):
        dfPPT.rename({datos_entidad_dfPPT.loc[n,2]:datos_entidad_dfPPT.loc[n,0]},axis=1,inplace=True)
    pqw=pd.merge(participantes_1_1_dfPPT,pqw)
    pqw=pd.merge(pqw,participantes_2_XZ[[0,1]],how='left',on=[0])
    pwq_prueba=pqw[pqw[1].isnull()]
    pqw[1]=pqw[0]
    return dfPPT,pwq_prueba, pqw,datos_entidad_dfPPT

In [80]:
dfPPT,pwq_prueba, pqw,datos_entidad_dfPPT=funayu1(dfPPT,participantes_2_XZ)

Aqui es una prueba, donde si pwq_prueba tiene elementos esos elemetos se deben de cambiar de nombre para que coincidan con la base

In [ ]:
pwq_prueba# se cambia o se busca por el nombre que debe ser

In [ ]:
#pqw.loc[9,1]='PAN_PRD_MC'# se cambia o se busca por el nombre que debe ser en participantes_2_XZ[0]
#pqw.loc[10,1]='PRI_PVEM_NA'# se cambia o se busca por el nombre que debe ser en participantes_2_XZ[0]
#pqw.loc[11,1]='PT_MORENA_ES'# se cambia o se busca por el nombre que debe ser en participantes_2_XZ[0]
pqw

In [83]:
#df11Ti_df22T['C_PRI_PRD_UDC']=0
#df11Ti_df22T['PRI']=0

In [84]:
def funayu2(dfPPT,datos_entidad_dfPPT,pqw,df11Ti_df22T):

    for n in range (0,len(pqw)):
        dfPPT.rename({pqw.loc[n,0]:pqw.loc[n,1]},axis=1,inplace=True)
    dfPPT[pqw[1]]=dfPPT[pqw[1]].replace(nan,0).astype('str')
    for py in range(0,len(pqw)):
        dfPPT[pqw[1][py]]=pd.to_numeric(dfPPT[pqw[1][py]].str.strip())
    dfPPT_2=dfPPT[pd.concat([datos_entidad_dfPPT[0],pqw[[1]].rename({1:0},axis=1)])[0]].reset_index(drop=True) 
    dfPPT_2=dfPPT_2.sort_values(datos_entidad_dfPPT[0][0],ascending=True)
    mun_xx=dfPPT[datos_entidad_dfPPT[0][0]].astype('int')
    df11Ti_df22T_2=df11Ti_df22T.set_index(datos_entidad_dfPPT[0][0])


    df11Ti_df22T_2=df11Ti_df22T_2.reset_index()

    dfPPT_2=dfPPT_2.sort_values(datos_entidad_dfPPT[0][0],ascending=True).reset_index(drop=True).set_index(datos_entidad_dfPPT[0][0])
    df11Ti_df22T_2=df11Ti_df22T_2.sort_values(datos_entidad_dfPPT[0][0],ascending=True).reset_index(drop=True).set_index(datos_entidad_dfPPT[0][0])

    for x in range(0,len(mun_xx)):

        for y in range(0,len(pqw[1])):

            c_x=mun_xx[x]
            c_y=pqw[1][y]

            if dfPPT_2.loc[c_x,c_y]>0:

                df11Ti_df22T_2.loc[c_x,c_y]=dfPPT_2.loc[c_x,c_y]
    df11Ti_df22T_2=df11Ti_df22T_2.reset_index()
    df11Ti_df22T_2=df11Ti_df22T_2.replace(0,nan)
    df11Ti_df22T=df11Ti_df22T_2.copy()

    return df11Ti_df22T,mun_xx,dfPPT_2

In [85]:
df11Ti_df22T,mun_xx,dfPPT_2=funayu2(dfPPT,datos_entidad_dfPPT,pqw,df11Ti_df22T)

In [ ]:
df11Ti_df22T

In [87]:


#def funayu3(dfPPT_2,df11Ti_df22T,mun_xx,datos_entidad_dfPPT):
if 1<2:

    if 'fallas1' in locals():
        del(fallas1)

    for yw in range(0,len(mun_xx)):
        dfPPT_2=dfPPT_2.reset_index()

        adf=pd.DataFrame(dfPPT_2[dfPPT_2[datos_entidad_dfPPT[0][0]]==mun_xx[yw]].replace(0,nan).dropna(axis=1).columns)
        adf=adf[adf[0]!=datos_entidad_dfPPT[0][0]].reset_index(drop=True)
        adf[1]=0
        adf[1]=adf[1].astype('object')
        adf[2]=adf[1].copy()
        adf[3]=adf[1].copy()
        dfPPT_2=dfPPT_2.set_index(datos_entidad_dfPPT[0][0])
        for t in range(0,len(adf)):
            a12=adf.loc[t,0]

            for t2 in range(0,len(tryu_XY_2)):
                a22=tryu_XY_2.loc[t2,0]

                if a12==a22:
                    adf.loc[t,1]=tryu_XY_2.loc[t2,1]
                    adf.loc[t,2]=len(adf.loc[t,1].split())

        lkj=pd.DataFrame(df11Ti_df22T[df11Ti_df22T[datos_entidad_dfPPT[0][0]]==mun_xx[yw]].replace(0,nan).dropna(axis=1).columns)
        lkj=lkj[lkj[0]!=datos_entidad_dfPPT[0][0]].reset_index(drop=True)
        lkj[1]=0
        lkj[1]=lkj[1].astype('object')
        lkj[2]=lkj[1].copy()
        lkj[3]=lkj[1].copy()
        for t in range(0,len(lkj)):
            a12=lkj.loc[t,0]
            

            for t2 in range(0,len(tryu_XY_2)):
                a22=tryu_XY_2.loc[t2,0]

                if a12==a22:
                    lkj.loc[t,1]=tryu_XY_2.loc[t2,1]
                    lkj.loc[t,2]=len(lkj.loc[t,1].split())


        for c2 in range(0,len(adf)):
            for c1 in range(0,len(lkj)):

                if set(adf.loc[c2,1].split())&set(lkj.loc[c1,1].split())==set(lkj.loc[c1,1].split()):
                    lkj.loc[c1,3]=1

        trope1=lkj[lkj[3]==1][0].reset_index(drop=True)
        trope0=lkj[lkj[3]==0][0].reset_index(drop=True)

        for pot in range(0,len(trope1)):
            df11Ti_df22T.loc[mun_xx[yw],trope1[pot]]=1
        for pot2 in range(0,len(trope0)):
            df11Ti_df22T.loc[mun_xx[yw],trope0[pot2]]=0  
        if len(trope0)>0:
            if 'fallas1' in locals():
                trope0_X=pd.DataFrame(lkj[lkj[3]==0][1].reset_index(drop=True))
                trope0_X[datos_entidad_dfPPT[0][0]]=mun_xx[yw]   
                fallas1=pd.concat([fallas1,trope0_X])
            else:

                trope0_X=pd.DataFrame(lkj[lkj[3]==0][1].reset_index(drop=True))
                trope0_X[datos_entidad_dfPPT[0][0]]=mun_xx[yw]
                fallas1=trope0_X

    #return fallas1

In [88]:
if 'fallas1' in locals():
    fallas1=fallas1.reset_index(drop=True)

    rutX=rut+'/' 
    if os.path.exists(rutX)==False:
        os.mkdir(rutX)

    rut1_X8=rutX+'datos'


    if os.path.exists(rut1_X8)==False:
        os.mkdir(rut1_X8)


    rut1_X8_1=rut1_X8+'/'

    rut1_X8_1=rut1_X8_1+'bases_procesadas'
    if os.path.exists(rut1_X8_1)==False:
        os.mkdir(rut1_X8_1)


    rut1_X8_1_x=rut1_X8_1+'/'+tipo_eleccion
    if os.path.exists(rut1_X8_1_x)==False:
        os.mkdir(rut1_X8_1_x)



    rut1_X8_1_x=rut1_X8_1_x+'/'+año
    if os.path.exists(rut1_X8_1_x)==False:
        os.mkdir(rut1_X8_1_x)
    rut1_X8_1_x=rut1_X8_1_x+'/'


    rut1_X8_1_x=rut1_X8_1_x+'/'+'errores 1'

    if os.path.exists(rut1_X8_1_x)==False:
        os.mkdir(rut1_X8_1_x)
    rut1_X8_1_x=rut1_X8_1_x+'/'

    filename2 =rut1_X8_1_x+'errores 1' +'.csv'  #cuando se separa por municipio
    fallas1.to_csv(filename2,index=False)

Aqui hace el procedimiento para todos municipios y al final se concatena el resultado

In [89]:
if "df11T_total" in locals():
    del(df11T_total)


for ID_eleccion in list3345[list3345.columns[0]]:
#for ID_eleccion in list3345[list3345.columns[0]]:
    

    if list3345.columns[0]=='distrito_l':
        df11Ti_df22T_list1=df11Ti_df22T[df11Ti_df22T['distrito_l']==ID_eleccion].dropna(axis=1)
        df11Ti=df11T[df11T['distrito_l']==ID_eleccion].reset_index(drop=True)
        df11Ti_df22T_list1=pd.DataFrame(df11Ti_df22T_list1.columns)
        df11Ti_df22T_list1=df11Ti_df22T_list1[df11Ti_df22T_list1[0]!='distrito_l']
        region='distritol_'+str(df11Ti['distrito_l'].unique()[0])
        ventil='ventilL'  

    if list3345.columns[0]=='municipio':
        df11Ti_df22T_list1=df11Ti_df22T[df11Ti_df22T['municipio']==ID_eleccion].dropna(axis=1)
        df11Ti=df11T[df11T['municipio']==ID_eleccion].reset_index(drop=True)
        df11Ti_df22T_list1=pd.DataFrame(df11Ti_df22T_list1.columns)
        df11Ti_df22T_list1=df11Ti_df22T_list1[df11Ti_df22T_list1[0]!='municipio']
        pruebah2=pd.DataFrame(df11Ti.columns)
        pruebah2=pruebah2[pruebah2[0]=='Nom_mun']
        if len(pruebah2)>=1:
            region='municipio_'+str(df11Ti['municipio'].unique()[0])+'_'+str(df11Ti['Nom_mun'].unique()[0])
        else:
            if len(df11Ti)>=1:
                region='municipio_'+str(df11Ti['municipio'].unique()[0])
        ventil='ventilL'  

    if list3345.columns[0]=='distrito_f':
        df11Ti_df22T_list1=df11Ti_df22T[df11Ti_df22T['distrito_f']==ID_eleccion].dropna(axis=1)
        df11Ti=df11T[df11T['distrito_f']==ID_eleccion].reset_index(drop=True)
        df11Ti_df22T_list1=pd.DataFrame(df11Ti_df22T_list1.columns)
        df11Ti_df22T_list1=df11Ti_df22T_list1[df11Ti_df22T_list1[0]!='distrito_f']
        region='distritof_'+str(df11Ti['distrito_f'].unique()[0])

    if list3345.columns[0]=='entidad':

        df11Ti_df22T_list1=df11Ti_df22T[df11Ti_df22T['entidad']==ID_eleccion].dropna(axis=1)
        df11Ti=df11T[df11T['entidad']==ID_eleccion].reset_index(drop=True)
        df11Ti_df22T_list1=pd.DataFrame(df11Ti_df22T_list1.columns)
        df11Ti_df22T_list1=df11Ti_df22T_list1[df11Ti_df22T_list1[0]!='entidad']
        pruebah2=pd.DataFrame(df11Ti.columns)
        pruebah2=pruebah2[pruebah2[0]=='Nom_ent']
        if len(pruebah2)>=1:
            region='entidad_'+str(df11Ti['entidad'].unique()[0])+'_'+str(df11Ti['Nom_ent'].unique()[0])
        else:
            if len(df11Ti)>=1:
                region='entidad_'+str(df11Ti['entidad'].unique()[0])    




    tryu=pd.merge(tryu_XY,df11Ti_df22T_list1).reset_index(drop=True)
    tryu_2=pd.merge(tryu_XY_2,df11Ti_df22T_list1).reset_index(drop=True)

    participantes_2,participantes_2_XZ_2N=mod_1.procesamiento_2_2(df11Ti,tryu_XY_2)



    df11Ti,datoseleccion_1,Cualiciones,participantes,Cualiciones_1=mod_1.procesamiento_3(df11Ti,participantes_2,sinCoa1X_2,datoseleccion_1_2,nombresst1_2,datos_entidad)
    df11Ti,partidos_porcen=mod_1.procesamiento_4(df11Ti,Cualiciones_1,Partidos,participantes) #separa en XX  

    if "df11T_total" in locals():

        df11T_total=pd.concat([df11T_total,df11Ti])

    else:
        df11T_total=df11Ti.copy()

se le pega la informacion de los ventiles

In [90]:
df11T_TP=pd.merge(gdf1[['entidad',  'seccion', 'ventilN', 'ventilL']]
                ,df11T_total,how='right',on=['entidad', 'seccion'])  

se acomodan un poco las variables

In [ ]:
df11Ti=df11T_TP.copy()
prueba3=pd.Series(df11Ti.columns).reset_index()
prueba1=pd.Series(df11Ti.columns)
prueba5=pd.Series(list(set(prueba1)-set(partidos_porcen))).reset_index() 
w0=pd.merge(prueba3,prueba5[[0]],how='inner',on=0)
w1=w0[0]
df11Ti=df11Ti[pd.concat([w1,partidos_porcen])]
df11Ti

esta parte genera una ruta para guardar el dataframe 

In [ ]:
rutX=rut+'/Pais/' 
if os.path.exists(rutX)==False:
    os.mkdir(rutX)

rut1=rutX+'datos'


if os.path.exists(rut1)==False:
    os.mkdir(rut1)


rut1_1=rut1+'/'

rut1_1=rut1_1+'bases_procesadas'
if os.path.exists(rut1_1)==False:
    os.mkdir(rut1_1)


rut1_1_x=rut1_1+'/'+tipo_eleccion
if os.path.exists(rut1_1_x)==False:
    os.mkdir(rut1_1_x)



rut1_1_x=rut1_1_x+'/'+año
if os.path.exists(rut1_1_x)==False:
    os.mkdir(rut1_1_x)
rut1_1_x=rut1_1_x+'/'
rut1_1_x  # ruta donde se va a guardar 

esta parte acomoda las calumnas del dataframe (de una forma estadarizada)

In [93]:
#filename2 =rut1_1_x+tipo_eleccion+'_elec_'+año+'_'+entidad+'_Municipio_'+str(nombre_municipio) +'.csv'  #cuando se separa por municipio
filename2 =rut1_1_x+tipo_eleccion.replace(' ','-')+'_elec_'+año+"_"+prep_AA+'_XXX.csv'
df11Ti.to_csv(filename2,index=False)

In [ ]:
filename2

In [ ]:
df11Ti

In [96]:
rt=pd.DataFrame(df11Ti.columns).copy()
rt[1]=0
for tr in range(0,len(rt)):
    a=rt.iloc[tr,0][-2:]
    b=rt.iloc[tr,0][-4:]
    if a=='**':
        rt.iloc[tr,1]=1
    if b=='**_%':
        rt.iloc[tr,1]=1
rt=rt[rt[1]==0].reset_index(drop=True)
df11Ti_2=df11Ti[rt[0]]

In [97]:
#filename2 =rut1_1_x+tipo_eleccion+'_elec_'+año+'_'+entidad+'_Municipio_'+str(nombre_municipio) +'.csv'  #cuando se separa por municipio
filename2 =rut1_1_x+tipo_eleccion.replace(' ','-')+'_elec_'+año+"_"+prep_AA+'_sumada.csv'
df11Ti_2.to_csv(filename2,index=False)

In [ ]:
filename2

In [ ]:
df11Ti.columns

In [ ]:
df11Ti[df11Ti[ 'Lista Nominal'].isnull()][[ 'entidad', 'seccion', 'ventilN', 'ventilL', 'municipio', 'Nom_mun',
       'Nom_ent', 'PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC', 'PAS', 'MORENA',
       'PES', 'PAN PRI PRD PAS', 'PAN PRI PRD', 'PAN PRI PAS', 'PAN PRD PAS',
       'PRI PRD PAS', 'PAN PRI', 'PAN PRD', 'PAN PAS', 'PRI PRD', 'PRI PAS',
       'PRD PAS', 'NUM VOTOS CAN NREG', 'NUM VOTOS NULOS', 'Lista Nominal']]

In [ ]:
df11Ti["Lista Nominal"]

In [ ]:
df11Ti[df11Ti[ 'Lista Nominal'].isnull()]

In [ ]:
aqui finaliza el programa


# Revisando el resultado de Sinaloa

In [ ]:
pd.set_option('display.max_rows', None)  # Muestra todas las filas
pd.set_option('display.max_columns', None)  # Muestra todas las columnas

dato1=df11Ti_2[df11Ti_2["municipio"]==1]

dato1.columns

In [ ]:
df11Ti_2.groupby([ 'municipio', 'Nom_mun',
       'Nom_ent'])[['PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC', 'PAS', 'MORENA',
       'PES', 'PAN PRI PRD PAS', 'PAN PRI PRD', 'PAN PRI PAS', 'PAN PRD PAS',
       'PRI PRD PAS', 'PAN PRI', 'PAN PRD', 'PAN PAS', 'PRI PRD', 'PRI PAS',
       'PRD PAS', 'NUM VOTOS CAN NREG', 'NUM VOTOS NULOS', 'Lista Nominal',
       'C PAN PRI PRD PAS', 'VT_calculado', 'Lista Nominal']].sum()

In [ ]:
lll=df11Ti_2.groupby([ 'municipio', 'Nom_mun',
       'Nom_ent'])[['PAN', 'PRI', 'PRD', 'PAS', 'PAN PRI PRD PAS', 'PAN PRI PRD', 'PAN PRI PAS', 'PAN PRD PAS',
       'PRI PRD PAS', 'PAN PRI', 'PAN PRD', 'PAN PAS', 'PRI PRD', 'PRI PAS',
       'PRD PAS', 'Lista Nominal','C PAN PRI PRD PAS', 'VT_calculado']].sum().reset_index()

lll.columns

In [ ]:
ll_l=lll[lll['municipio']==11]


ll_l[['PAN', 'PRI', 'PRD', 'PAS', 'PAN PRI PRD PAS', 'PAN PRI PRD',
       'PAN PRI PAS', 'PAN PRD PAS', 'PRI PRD PAS', 'PAN PRI', 'PAN PRD',
       'PAN PAS', 'PRI PRD', 'PRI PAS', 'PRD PAS']].sum().sum()

In [ ]:
5806 +10738 +956+7938

In [ ]:
abriendo el primer dato

In [37]:
sss="/Users/ds/Documents/Base_municipales_200924/Pais/datos/bases_procesadas/Municipal/2024/Sinaloa/Municipal_elec_2024_Sinaloa_sumada.csv"
base222=pd.read_csv(sss, header=0)


In [ ]:
base222.groupby([ 'municipio', 'Nom_mun',
       'Nom_ent'])[['PAN', 'PRI', 'PRD', 'PT', 'PVEM', 'MC', 'PAS', 'MORENA',
       'PES', 'PAN PRI PRD PAS', 'PAN PRI PRD', 'PAN PRI PAS', 'PAN PRD PAS',
       'PRI PRD PAS', 'PAN PRI', 'PAN PRD', 'PAN PAS', 'PRI PRD', 'PRI PAS',
       'PRD PAS', 'NUM VOTOS CAN NREG', 'NUM VOTOS NULOS', 'Lista Nominal',
       'C PAN PRI PRD PAS', 'VT_calculado', 'Lista Nominal']].sum()

In [ ]:
863666-352760

In [ ]:
cgfy

# Creando las bases de datos de cada municipio del estado


In [ ]:
df11T.columns

In [ ]:
# Supongamos que df11T_municipio es un DataFrame de pandas
lista_municipios = []
for i in df11T_municipio['Nom_mun'].unique():
    lista_municipios.append(i)  # Usa paréntesis

# Ahora, supongamos que quieres encontrar el máximo de una columna específica
# Por ejemplo, digamos que tienes una columna 'valor' en df11T_municipio

maximos = {}
for mun in lista_municipios:
    maximo = df11T_municipio[df11T_municipio['Nom_mun'] == mun]['valor'].max()
    maximos[mun] = maximo

print(maximos)  # Muestra el máximo por cada municipio


In [ ]:
municipios=[]
independiente=['CAND_IND_1', 'CAND_IND_2', 'CAND_IND_3',
       'CAND_IND_4', 'CAND_IND_5', 'CAND_IND_6', 'CAND_IND_7', 'CAND_IND_8',
       'CAND_IND_9']





for no_municipio in df11T['municipio'].unique():
    df11T_municipio = df11T[df11T["municipio"]==no_municipio]
    for ind in df11T_municipio[independiente]:
        suma=df11T_municipio[ind].sum()
        if suma==0:
            df11T_municipio = df11T_municipio.drop(ind, axis=1)
            

    #Hasta este punto ya se eliminaron los candidatos que estan vacios en las elecciones
    frecuencia = df11T_municipio['Nom_mun'].value_counts()

    # Obtener el municipio más frecuente
    palabra_frecuente = frecuencia.idxmax()
    nombre_municipio=palabra_frecuente.replace(" ", "_")
    titulo=tipo_eleccion+"_elec_"+año+"_"+nombre_municipio+"_"




    print(titulo)

In [ ]:
len(municipios)

In [ ]:
df_entidad=[]

for nom_estado in df11T['Nom_ent'].unique():
    #clasificando por estado/entidad
    df_estado = df11T[df11T["Nom_ent"]== nom_estado]
    region='Estados Unidos Mexicanos'
    titulo=('Elecciones: '+tipo_eleccion+' '+año +'\n'+region.replace('_',' ').split()[0][0].upper()+region.replace('_',' ').split()[0][1:]
               +' '+region.replace('_',' ').split()[1]+  ' '  
               +    ' '.join(region.replace('_',' ').split()[2:] )+ "\n" + nom_estado)      #### con este se cambia el titulo en la grafica
           
            #modificando el titulo agragando el id del distriti_f
    titulo=titulo.replace('_',' ')

            ####################### Limpiando el titulo de la variable df
    x_clean=nom_estado.replace(" ", "_").replace(".", "").replace("/", "_")
    df11T_df_Cabecera= f"df11T_{x_clean}"  #De esta forma se llamaran las variables
            #creando la variable dinamica con el titulo correspondiente a las iteraciones del for
    globals()[df11T_df_Cabecera] = df_estado

    df_entidad.append(f"df11T_{x_clean}")


    df11T_df_Cabecera1=globals()[df11T_df_Cabecera]


            
    rutk2=rut
    rutXX_2=rutk2+'/Pais/' 
    rut4=rutXX_2+'graficas'
    if os.path.exists(rut4)==False:
        os.mkdir(rut4)
    rut4_2=rut4+'/'+tipo_eleccion
    if os.path.exists(rut4_2)==False:
        os.mkdir(rut4_2)
    rut4_2_x=rut4_2+'/'+año
    if os.path.exists(rut4_2_x)==False:
        os.mkdir(rut4_2_x) 
    rut4_2_x=rut4_2_x+'/'

    rut_CV=rut4_2_x+tipo_eleccion.replace(' ','-')+'_elec_'+año+'_'+str(region).replace(' ','-')


    fig_ventiles = grafica_ventiles(df11T_df_Cabecera1, titulo)
    graf1=rut_CV+"_"+x_clean+"_"+'_ventiles.png'   ####Aqui se modifica el nombre para guardar 
    fig_ventiles.savefig(fname=graf1)
    fig_ventiles.close()

    fig_participacion= grafica_participacion(df11T_df_Cabecera1, titulo)  #regresa  fig_participacion
    graf2=rut_CV+"_"+x_clean+"_"+'_participacion.png'
    fig_participacion.savefig(fname=graf2)
    fig_participacion.close()

    fig_resultados=grafica_resultados(df11T_df_Cabecera1,titulo)  #regresa fig_resultados
    graf3=rut_CV+"_"+x_clean+"_"+'_resultados.png'
    fig_resultados.savefig(fname=graf3)
    fig_resultados.close()
            


print(df_entidad)

In [ ]:
independiente=['CAND_IND_1', 'CAND_IND_2', 'CAND_IND_3',
       'CAND_IND_4', 'CAND_IND_5', 'CAND_IND_6', 'CAND_IND_7', 'CAND_IND_8',
       'CAND_IND_9']


for ind in df11T[independiente]:
    suma=df11T[ind].sum()
    print(suma)

In [ ]:
sg

# Municipios locales partidos politicos

In [ ]:
# empatando la base con la de diputados federales  
Ruta_base_prep="/Users/ds/Documents/DL_Ayuntamientos/Pais/datos/bases_procesadas/Diputados Locales/2024/Diputados-Locales_elec_2024_DIP MR_XXX.csv"
df11T=pd.read_csv(Ruta_base_prep,header=0)
df11T.columns

In [ ]:
df11T_mun=df11T[df11T["municipio"]==10]
df11T_mun

In [ ]:
rut="/Users/ds/Documents/Base_municipales_200924/Albaro_Obregon"
#tipo_eleccion='Municipal'  # de que fue la eleccion 
#tipo_eleccion='Diputados Federales' 
tipo_eleccion='Diputados Locales'
#tipo_eleccion='Presidencial'  
#tipo_eleccion='Gobernatura' 
#tipo_eleccion='Senaduria'
#tipo_eleccion="Jefa de Gobierno"
#clave_entidad=15

año='2024'                  #año de la eleccion  

prep_AA='ALVARO_OBREGON'

In [ ]:
rutX=rut+'/Pais/' 
if os.path.exists(rutX)==False:
    os.mkdir(rutX)

rut1=rutX+'datos'


if os.path.exists(rut1)==False:
    os.mkdir(rut1)


rut1_1=rut1+'/'

rut1_1=rut1_1+'base_11_sep_2024'
if os.path.exists(rut1_1)==False:
    os.mkdir(rut1_1)


rut1_1_x=rut1_1+'/'+tipo_eleccion
if os.path.exists(rut1_1_x)==False:
    os.mkdir(rut1_1_x)



rut1_1_x=rut1_1_x+'/'+año
if os.path.exists(rut1_1_x)==False:
    os.mkdir(rut1_1_x)
rut1_1_x=rut1_1_x+'/'
rut1_1_x  # ruta donde se va a guardar 

In [ ]:
#filename2 =rut1_1_x+tipo_eleccion+'_elec_'+año+'_'+entidad+'_Municipio_'+str(nombre_municipio) +'.csv'  #cuando se separa por municipio
filename2 =rut1_1_x+tipo_eleccion.replace(' ','-')+'_elec_'+año+"_"+prep_AA+'_XXX.csv'
df11T_mun.to_csv(filename2,index=False)

In [ ]:
df11Ti=df11T_mun.copy()

In [ ]:
rt=pd.DataFrame(df11Ti.columns).copy()
rt[1]=0
for tr in range(0,len(rt)):
    a=rt.iloc[tr,0][-2:]
    b=rt.iloc[tr,0][-4:]
    if a=='**':
        rt.iloc[tr,1]=1
    if b=='**_%':
        rt.iloc[tr,1]=1
rt=rt[rt[1]==0].reset_index(drop=True)
df11Ti_2=df11Ti[rt[0]]


#filename2 =rut1_1_x+tipo_eleccion+'_elec_'+año+'_'+entidad+'_Municipio_'+str(nombre_municipio) +'.csv'  #cuando se separa por municipio
filename2 =rut1_1_x+tipo_eleccion.replace(' ','-')+'_elec_'+año+"_"+prep_AA+'_sumada.csv'
df11Ti_2.to_csv(filename2,index=False)

In [ ]:
#filename2 =rut1_1_x+tipo_eleccion+'_elec_'+año+'_'+entidad+'_Municipio_'+str(nombre_municipio) +'.csv'  #cuando se separa por municipio
filename2 =rut1_1_x+tipo_eleccion.replace(' ','-')+'_elec_'+año+"_"+prep_AA+'_sumada.csv'
df11Ti_2.to_csv(filename2,index=False)

In [ ]:
df11T_mun=df11T[(df11T["municipio"]==10)] # & (df11T["municipio"]==10)]
df11T_mun

In [ ]:
df11T.columns

In [ ]:
df11T[['PAN', 'PRI', 'PRD', 'PVEM', 'PT',
       'MC', 'MORENA', 'PAN PRI PRD', 'PAN PRI', 'PAN PRD', 'PRI PRD',
       'PVEM PT MORENA', 'CAND IND 1', 'NUM VOTOS CAN NREG', 'NUM VOTOS NULOS',
       'Lista Nominal', 'C PAN PRI PRD', 'C PVEM PT MORENA']].sum()

# Pagando la lista de minicipios

In [ ]:
Ruta_base_prep="/Users/ds/Documents/DL_Ayuntamientos/Pais/datos/bases_procesadas/Presidencial/2024/Presidencial_elec_2024_20240608_2030_COMPUTOS_PRES_XXX.csv"
df11T=pd.read_csv(Ruta_base_prep,header=0)
df11T.columns

In [ ]:
df11T["Nom_ent"].unique()

In [ ]:
df11T=df11T[df11T["Nom_ent"]=='MÉXICO']
df11T

In [ ]:
Ruta_base_prep2="/Users/ds/Documents/DL_Ayuntamientos/Pais/datos/base_11_sep_2024/Municipal/2024/Municipal_elec_2024_NAUCALPAN_XXX.csv"
df11T_secciones=pd.read_csv(Ruta_base_prep2,header=0)
df11T_secciones.columns

In [ ]:
df11T_secciones[['seccion', 'municipio','Nom_mun']]

In [ ]:
df11T_TP=pd.merge(df11T,df11T_secciones[['seccion', 'municipio','Nom_mun']],how='inner',on=['seccion'])  

In [ ]:
df11T_TP.columns

In [ ]:
df11Tii=df11T_TP[['entidad', 'seccion', 'ventilN', 'ventilL', 'municipio','distrito_f','Nom_mun', 'Nom_ent',
       'DF_Cabecera', 'PAN', 'PRI', 'PRD', 'PVEM', 'PT', 'MC', 'MORENA',
       'PAN PRI PRD', 'PAN PRI', 'PAN PRD', 'PRI PRD', 'PVEM PT MORENA',
       'PVEM PT', 'PVEM MORENA', 'PT MORENA', 'NUM VOTOS CAN NREG',
       'NUM VOTOS NULOS', 'Lista Nominal', 'C PAN PRI PRD', 'C PVEM PT MORENA',
       'CVE', 'NCVE', 'VT_calculado', 'ganador', 'participacion', 'PAN_**',
       'PRI_**', 'PRD_**', 'PVEM_**', 'PT_**', 'MORENA_**', 'C PAN PRI PRD_%',
       'C PVEM PT MORENA_%', 'MC_%', 'NUM VOTOS CAN NREG_%',
       'NUM VOTOS NULOS_%', 'PAN_**_%', 'PRI_**_%', 'PRD_**_%', 'PVEM_**_%',
       'PT_**_%', 'MORENA_**_%']]
df11Tii

In [ ]:
rut='/Users/ds/Documents/DL_Ayuntamientos'    #Ruta donde se guardan las bases
#tipo_eleccion='Municipal'  # de que fue la eleccion 
#tipo_eleccion='Diputados Federales' 
#tipo_eleccion='Diputados Locales'
tipo_eleccion='Presidencial'  
#tipo_eleccion='Gobernatura' 
#tipo_eleccion='Senaduria'

#clave_entidad=15

año='2024'                  #año de la eleccion  

prep_AA='NAUCALPAN'

In [ ]:
rutX=rut+'/Pais/' 
if os.path.exists(rutX)==False:
    os.mkdir(rutX)

rut1=rutX+'datos'


if os.path.exists(rut1)==False:
    os.mkdir(rut1)


rut1_1=rut1+'/'

rut1_1=rut1_1+'base_11_sep_2024'
if os.path.exists(rut1_1)==False:
    os.mkdir(rut1_1)


rut1_1_x=rut1_1+'/'+tipo_eleccion
if os.path.exists(rut1_1_x)==False:
    os.mkdir(rut1_1_x)



rut1_1_x=rut1_1_x+'/'+año
if os.path.exists(rut1_1_x)==False:
    os.mkdir(rut1_1_x)
rut1_1_x=rut1_1_x+'/'
rut1_1_x  # ruta donde se va a guardar 

In [ ]:
#filename2 =rut1_1_x+tipo_eleccion+'_elec_'+año+'_'+entidad+'_Municipio_'+str(nombre_municipio) +'.csv'  #cuando se separa por municipio
filename2 =rut1_1_x+tipo_eleccion.replace(' ','-')+'_elec_'+año+"_"+prep_AA+'_XXX.csv'
df11Tii.to_csv(filename2,index=False)

In [ ]:
filename2

In [ ]:
import pandas as pd
ruta="/Users/ds/Documents/Base_municipales_200924/Pais/datos/bases_procesadas/Municipal/2024/Guanajuato/Municipal_elec_2024_09062024_1200_COMPUTOS_AYUN_GTO_XXX.csv"
df11T= pd.read_csv(ruta)
df11T

In [ ]:
df11T[df11T["municipio"]==46][['PAN', 'PRI', 'PRD','PAN PRI PRD', 'PAN PRI', 'PAN PRD', 'PRI PRD']].sum().sum()

In [ ]:
df11T.columns